# Number theory neat examples

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 1 : </span>
<span style="font-size: 16pt; font-style: italic;">Ulam spiral, modular exponentiation</span>

Anton Antonov    
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
January 2025     

----

## Introduction

-----

## Setup

In [43]:
use Math::NumberTheory;
use Math::NumberTheory::Utilities;

In [44]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [45]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

In [46]:
multi sub highlight-html-table(Str:D $s, @highlight, Str:D :$color = 'Orange', :$font-size = Whatever, :$font-weight = 'normal') { 
    return highlight-html-table($s, :@highlight, :$color, :$font-size);
} 

multi sub highlight-html-table(Str:D $s, :h(:@highlight)!, Str:D :c(:$color) = 'Orange', :s(:$font-size) = Whatever, :w(:$font-weight) = 'normal') { 
    my $head = $font-size ~~ Numeric:D ?? "<span style=\"color: $color; font-size:{$font-size}pt; font-weight:$font-weight\">" !! "<span style=\"color: $color; font-weight:$font-weight\">";
    reduce( 
        { $^a.subst( / <?after '<td>'> $^b <?before '</td>'> /, $head ~ $^b ~ '</span>', :g) }, 
        $s, |@highlight) 
}

&highlight-html-table

------

## Ulam spiral

Spiral lattice with highlighted primes:

In [47]:
&spiral-lattice.signature

(Int:D $n where { ... }, :l(:end-corner(:$last-at)) is copy = Whatever)

In [48]:
#% html
my $n = 11;
spiral-lattice($n, end-corner => 'bottom-right')
==> { $_.map(*.kv.Hash) }()
==> to-html(field-names => (^$n)».Str)
==> highlight-html-table(h => (1..prime-pi($n²))».&prime».Str, c => 'Orange', w => 'bold')

0,1,2,3,4,5,6,7,8,9,10
101,100,99,98,97,96,95,94,93,92,91
102,65,64,63,62,61,60,59,58,57,90
103,66,37,36,35,34,33,32,31,56,89
104,67,38,17,16,15,14,13,30,55,88
105,68,39,18,5,4,3,12,29,54,87
106,69,40,19,6,1,2,11,28,53,86
107,70,41,20,7,8,9,10,27,52,85
108,71,42,21,22,23,24,25,26,51,84
109,72,43,44,45,46,47,48,49,50,83
110,73,74,75,76,77,78,79,80,81,82


Ulam spiral table:

In [49]:
#% html
spiral-lattice($n)
==> { $_.deepmap({ is-prime($_) ?? $_ !! '' }) }()
==> { $_.map(*.kv.Hash) }()
==> to-html(field-names => (^$n)».Str)

0,1,2,3,4,5,6,7,8,9,10
101,,,,97,,,,,,
,,,,,61,,59,,,
103,,37,,,,,,31,,89
,67,,17,,,,13,,,
,,,,5,,3,,29,,
,,,19,,,2,11,,53,
107,,41,,7,,,,,,
,71,,,,23,,,,,
109,,43,,,,47,,,,83
,73,,,,,,79,,,


In [50]:
#% js
my @mat = spiral-lattice(101).deepmap({ is-prime($_) ?? $_ !! 0 });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Viridis', :!tooltip)

Diagonals for $4 x^2 - 2 x + 41$:

In [51]:
#% js
my $n = 201;
my @highlight = (1..$n).map({ 4 * $_ ** 2 - 2 * $_ + 41 })».Int;
my @mat = spiral-lattice($n).deepmap({ is-prime($_) ?? ($_ ∈ @highlight ?? 3 !! 1) !! 0 });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Viridis', :!tooltip)

-----

## Next prime

The function `next-prime(x)` gives the smallest prime above $x$.

Highlight the sequence of primes using `next-prime`:

In [52]:
#% html
my $n = 70;

my @highlight = (1..$n).map({ next-prime($_ - 1) == $_ ?? $_ !! Empty })».Str;

(1...$n)
==> to-html(:10multi-column)
==> highlight-html-table(:@highlight, c => 'Red', w => 'bold')

1,8,15,22,29,36,43,50,57,64
2,9,16,23,30,37,44,51,58,65
3,10,17,24,31,38,45,52,59,66
4,11,18,25,32,39,46,53,60,67
5,12,19,26,33,40,47,54,61,68
6,13,20,27,34,41,48,55,62,69
7,14,21,28,35,42,49,56,63,70


Visualize the [sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for the first 5 primes:

In [53]:
#% js
my @data = 
(0,0.1...30).map( -> $x { 
                         (1..10).map( -> $n { %( 
                                group => next-prime($n), 
                                :$x, 
                                y => next-prime($n) * sin($x * π / next-prime($n))
                            ) 
                        }) 
                }).flat(1);

js-d3-list-line-plot(@data, background => 'none')

Plot a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) of the distance to the next prime:

In [54]:
#% js
my @mat = spiral-lattice(101).deepmap({ next-prime($_) - $_ });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Turbo', :!tooltip)

Sunflower seed representation of `next-prime`:

In [55]:
my @sunflower = (1..15000).map({ .&next-prime }).map({
    my $a = $_;
    my $angle = ((2 * π * $a) / (ϕ ** 2));
    [ sqrt($a) * cos($angle), sqrt($a) * sin($angle) ]
});

deduce-type(@sunflower)

Vector(Vector(Atom((Numeric)), 2), 15000)

In [56]:
#% js
js-d3-list-plot(@sunflower, 
    background => 'none',
    point-size => 3,
    stroke-color => 'PaleGoldenrod', 
    width => 400, height => 400, 
    :!axes, 
)

----

## Modular exponentiation (power mod)

The function `power-mod(a, b, m)` gives $a^b \mod m$.

Plot a list of powers of 3 where the exponent is varied, modulo some prime number:

In [57]:
#%js
my @a = (1 ... (prime(44)-1) ).map({ power-mod(3, $_ , prime(44)) });
js-d3-list-plot(@a, 
    title-color => 'DimGray',
    plot-label => "3ᵇ mod {prime(44)}",
    background => 'none',
    width => 700,
    height => 160
)

Plot values of varying powers of numbers with a fixed modulus:

In [58]:
#%js
my @a = ((2..100) X (2..100)).map({ power-mod($_.head, $_.tail, 32) }).rotor(99);
js-d3-matrix-plot(@a, width => 400, height => 400, :!grid-lines)

Plot an a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) where numbers are colored based on `power-mod`:

In [59]:
#% js
my @mat = spiral-lattice(101).deepmap({ power-mod($_, 3, 17)});
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Rainbow')

----

## Modular inverse

The function `modular-inverse(k,n)` gives the modular inverse of $k$ modulo $n$.

- `modular-inverse` is also known as modular multiplicative inverse.
- Typically used in modular arithmetic and cryptography.
- `modular-inverse(k,n)` gives the number $r$ such that the remainder of the division of $r k$ by $n$ is equal to $1$.
- If $k$ and $n$ are not coprime, no modular inverse exists and `modular-inverse(k,n)` returns `Nil`.

In [60]:
#%html
my @row = 2, 3, 5, 7;
my @col = 1..10;
(@row X @col).map({ $_.tail => modular-inverse($_.tail, $_.head) // '' }).rotor(@col.elems)».Hash
==> { $_.kv.map(-> $k, %v { %v<n> = @row[$k]; %v }) }()
==> to-html(field-names => ['n', |@col».Str])

n,1,2,3,4,5,6,7,8,9,10
2,1,,1,,1,,1,,1,
3,1,2,,1,2,,1,2,,1
5,1,3,2,4,,1,3,2,4,
7,1,4,5,2,3,6,,1,4,5


Visualize when a number is invertible modulo 12:

In [61]:
#% js
my $n = 10;
my @a = ((1..$n) X (1..$n)).map({ not so modular-inverse($_.head + $_.tail ** 2, 12) })».Int.rotor($n);

js-d3-matrix-plot(@a, width => 400, height => 400, :!grid-lines, color-palette => 'Blues', :!tooltip)